<a href="https://colab.research.google.com/github/blossomsu/CASA0025Building_Spatial_Applications_with_Big_Data/blob/main/Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Review

### W1 Data Import

1. 数据来源
如果是database：直接将database与DuckDBconnect; 如果是csv（等等）：新建空白database，create table

2. sqlmagic OR config.()

In [8]:
%pip install duckdb leafmap

#Library import
import duckdb
import leafmap #做交互式地图的 Python 库
import pandas as pd

con = duckdb.connect() #创建一个 DuckDB 连接对象（启动了一个临时数据库实例）
#把这个数据库的“连接对象”存在变量 con 里
#之后你对 con 做的所有操作，都是在跟 DuckDB 说话

#让 DuckDB 可以直接读网络上的文件
con.install_extension("httpfs")
con.load_extension("httpfs")

#给 DuckDB 加“空间数据类型 + 空间函数”
con.install_extension("spatial")
con.load_extension("spatial")

#### Downloading Data

In [9]:
url = "https://storage.googleapis.com/qm2/CASA0025/cities.zip"
leafmap.download_file(url, unzip=True, overwrite=True)

Downloading...
From: https://storage.googleapis.com/qm2/CASA0025/cities.zip
To: /content/cities.zip
100%|██████████| 214k/214k [00:00<00:00, 59.6MB/s]

Extracting files...


'/content/cities.zip'

In [55]:
con.sql("""
CREATE TABLE countries AS
SELECT *
FROM 'https://storage.googleapis.com/qm2/CASA0025/countries.csv';
""")

In [10]:
con.sql("SHOW TABLES;")

┌─────────┐
│  name   │
│ varchar │
├─────────┤
│ 0 rows  │
└─────────┘

In [11]:
con.sql("SHOW DATABASES;")

┌───────────────┐
│ database_name │
│    varchar    │
├───────────────┤
│ memory        │
└───────────────┘

In [12]:
url = "https://storage.googleapis.com/qm2/CASA0025/nyc_data.db.zip"
leafmap.download_file(url, unzip=True)

nyc_data.db.zip already exists. Skip downloading. Set overwrite=True to overwrite.


'/content/nyc_data.db.zip'

In [13]:
con1 = duckdb.connect("nyc_data.db")

con1.install_extension("httpfs")
con1.load_extension("httpfs")

con1.install_extension("spatial")
con1.load_extension("spatial")

In [14]:
con1.sql("SHOW TABLES;")

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ nyc_census_blocks   │
│ nyc_homicides       │
│ nyc_neighborhoods   │
│ nyc_streets         │
│ nyc_subway_stations │
└─────────────────────┘

In [15]:
con1.sql("SHOW DATABASES;")

┌───────────────┐
│ database_name │
│    varchar    │
├───────────────┤
│ nyc_data      │
└───────────────┘

In [28]:
con1.sql("DESCRIBE nyc_census_blocks;")

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │ column_type │  null   │   key   │ default │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ BLKID       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ POPN_TOTAL  │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ POPN_WHITE  │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ POPN_BLACK  │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ POPN_NATIV  │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ POPN_ASIAN  │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ POPN_OTHER  │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ BORONAME    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ geom        │ GEOMETRY    │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

In [29]:
con1.sql("DESCRIBE nyc_neighborhoods;")

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │ column_type │  null   │   key   │ default │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ BORONAME    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ NAME        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ geom        │ GEOMETRY    │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

#### Reading Data

In [16]:
con.read_csv('cities.csv')
#con.read_csv('cities.csv', parallel=True) #让 DuckDB 多线程读 CSV，对大文件更快

#con.read_json('cities.json')
#con.read_parquet('cities.parquet')

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │
│ int64 │     varchar      │ varchar │  double   │  double   │   int64    │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │
│     6 │ Mariehamn        │ ALD     │    60.097 │    19.949 │      10682 │
│     7 │ Ramallah         │ PSE     │  31.90294 │  35.20621 │      24599 │
│     8 │ Vatican City     │ VAT     │  41.90001 │  12.44781 │        832 │
│     9 │ Poitier          │ FRA     │  46.58329 │   0.33328 │      85960 │
│    10 │ Cl

In [17]:
con.sql("FROM ST_Read('cities.shp')")
#con.sql("FROM ST_Read('cities.geojson')")

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┬─────────────────────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │            geom             │
│ int64 │     varchar      │ varchar │  double   │  double   │   int64    │          geometry           │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┼─────────────────────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │ POINT (32.5333 0.5833)      │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │ POINT (30.275 0.671)        │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │ POINT (15.799 40.642)       │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │ POINT (14.656 41.563)       │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │ POINT (7.315 45.737)        │
│     6 │ Mariehamn        │ ALD     │    60.0

In [18]:
con.sql("SELECT * FROM 'cities.csv'")

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │
│ int64 │     varchar      │ varchar │  double   │  double   │   int64    │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │
│     6 │ Mariehamn        │ ALD     │    60.097 │    19.949 │      10682 │
│     7 │ Ramallah         │ PSE     │  31.90294 │  35.20621 │      24599 │
│     8 │ Vatican City     │ VAT     │  41.90001 │  12.44781 │        832 │
│     9 │ Poitier          │ FRA     │  46.58329 │   0.33328 │      85960 │
│    10 │ Cl

In [19]:
con.sql("CREATE TABLE cities AS SELECT * FROM ST_Read('cities.geojson')")
#把“空间文件的数据”真正存进了 DuckDB,从“分析引擎模式” → “数据库模式”的切换

In [41]:
con.sql("""
SELECT *
FROM cities
LIMIT 5
""").df()

,id,name,country,latitude,longitude,population,geom
0,1,Bombo,UGA,0.5833,32.5333,75000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,2,Fort Portal,UGA,0.6710,30.2750,42670,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,3,Potenza,ITA,40.6420,15.7990,69060,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,4,Campobasso,ITA,41.5630,14.6560,50762,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,5,Aosta,ITA,45.7370,7.3150,34062,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [21]:
con.sql("""
DESCRIBE cities
""")

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │ column_type │  null   │   key   │ default │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ id          │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ name        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ country     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ latitude    │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ longitude   │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ population  │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ geom        │ GEOMETRY    │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

#### Data Frames

In [22]:
df = pd.read_csv('cities.csv')
df

,id,name,country,latitude,longitude,population
0,1,Bombo,UGA,0.58330,32.53330,75000
1,2,Fort Portal,UGA,0.67100,30.27500,42670
2,3,Potenza,ITA,40.64200,15.79900,69060
3,4,Campobasso,ITA,41.56300,14.65600,50762
4,5,Aosta,ITA,45.73700,7.31500,34062
...,...,...,...,...,...,...
1244,1245,Rio de Janeiro,BRA,-22.92502,-43.22502,11748000
1245,1246,Sao Paulo,BRA,-23.55868,-46.62502,18845000
1246,1247,Sydney,AUS,-33.92001,151.18518,4630000
1247,1248,Singapore,SGP,1.29303,103.85582,5183700


### W2 SQL Basic

#### Column/Row Filtering

In [23]:
con.sql("""
SELECT name, country
FROM cities
WHERE country = 'CHN'
LIMIT 5
""")

┌──────────┬─────────┐
│   name   │ country │
│ varchar  │ varchar │
├──────────┼─────────┤
│ Macau    │ CHN     │
│ Xiamen   │ CHN     │
│ Nanchong │ CHN     │
│ Neijiang │ CHN     │
│ Nanyang  │ CHN     │
└──────────┴─────────┘

#### Sorting Results

In [24]:
con.sql("""
SELECT name, country
FROM cities
WHERE country = 'CHN'
ORDER BY name ASC
""")
#DESC

┌───────────┬─────────┐
│   name    │ country │
│  varchar  │ varchar │
├───────────┼─────────┤
│ Altay     │ CHN     │
│ Ankang    │ CHN     │
│ Anshan    │ CHN     │
│ Baoshan   │ CHN     │
│ Baotou    │ CHN     │
│ Beian     │ CHN     │
│ Beijing   │ CHN     │
│ Bose      │ CHN     │
│ Changchun │ CHN     │
│ Changsha  │ CHN     │
│    ·      │  ·      │
│    ·      │  ·      │
│    ·      │  ·      │
│ Yinchuan  │ CHN     │
│ Yining    │ CHN     │
│ Yueyang   │ CHN     │
│ Yulin     │ CHN     │
│ Yulin     │ CHN     │
│ Zaozhuang │ CHN     │
│ Zhangye   │ CHN     │
│ Zhanjiang │ CHN     │
│ Zhengzhou │ CHN     │
│ Zibo      │ CHN     │
├───────────┴─────────┤
│ 100 rows  2 columns │
│ (20 shown)          │
└─────────────────────┘

#### Unique Values

In [25]:
con.sql("""
SELECT DISTINCT country
FROM cities
ORDER BY name ASC
""")

┌────────────┐
│  country   │
│  varchar   │
├────────────┤
│ ARE        │
│ GHA        │
│ USA        │
│ LBY        │
│ SYR        │
│ JOR        │
│ AND        │
│ DZA        │
│ MDG        │
│ WSM        │
│  ·         │
│  ·         │
│  ·         │
│ SYC        │
│ AUT        │
│ LAO        │
│ LTU        │
│ NLD        │
│ PRK        │
│ IRN        │
│ ARM        │
│ HRV        │
│ PHL        │
├────────────┤
│  200 rows  │
│ (20 shown) │
└────────────┘

#### Aggregating Data

In [47]:
con.sql("""
SELECT COUNT(*) FROM cities;
""")

┌─────────────────────────┐
│ count(DISTINCT country) │
│          int64          │
├─────────────────────────┤
│                     200 │
└─────────────────────────┘

In [26]:
con.sql("""
SELECT country, COUNT(name) AS num
FROM cities
GROUP BY country
ORDER BY num
""")

┌─────────┬───────┐
│ country │  num  │
│ varchar │ int64 │
├─────────┼───────┤
│ GHA     │     1 │
│ HTI     │     1 │
│ TJK     │     1 │
│ VAT     │     1 │
│ CZE     │     1 │
│ LBR     │     1 │
│ LIE     │     1 │
│ IRL     │     1 │
│ BRB     │     1 │
│ MHL     │     1 │
│  ·      │     · │
│  ·      │     · │
│  ·      │     · │
│ MEX     │    27 │
│ FRA     │    30 │
│ AUS     │    36 │
│ ATA     │    40 │
│ CAN     │    45 │
│ BRA     │    46 │
│ IND     │    69 │
│ RUS     │    81 │
│ CHN     │   100 │
│ USA     │   114 │
├─────────┴───────┤
│    200 rows     │
│   (20 shown)    │
└─────────────────┘

In [49]:
con.sql("""
SELECT MAX(population) FROM cities;
""")

┌─────────────────┐
│ max(population) │
│      int32      │
├─────────────────┤
│        35676000 │
└─────────────────┘

In [50]:
con.sql("""
SELECT SUM(population) FROM cities;
""")

┌─────────────────┐
│ sum(population) │
│     int128      │
├─────────────────┤
│      1475534501 │
└─────────────────┘

In [51]:
con.sql("""SELECT AVG(population) FROM cities;""")

┌────────────────────┐
│  avg(population)   │
│       double       │
├────────────────────┤
│ 1181372.6989591673 │
└────────────────────┘

#### Joining Tables

In [40]:
con1.sql("""
SELECT DISTINCT c.boroname, n.name
FROM nyc_census_blocks c
LEFT JOIN nyc_neighborhoods n
ON c.boroname = n.boroname
LIMIT 15
""").df()

,BORONAME,NAME
0,Staten Island,Rosebank
1,Staten Island,Westerleigh-Castleton
2,Brooklyn,Carroll Gardens
3,Brooklyn,Bedford-Stuyvesant
4,Manhattan,Garment District
5,Brooklyn,Fort Green
6,Manhattan,Clinton
7,Brooklyn,Bushwick
8,Staten Island,Ardon Heights
9,Manhattan,Washington Heights


#### String Manipulation

In [43]:
con1.sql("""
SELECT DISTINCT NAME
FROM nyc_subway_stations
WHERE NAME LIKE '%St%';
""")

┌───────────────┐
│     NAME      │
│    varchar    │
├───────────────┤
│ 138th St      │
│ 167th St      │
│ 170th St      │
│ 174th St      │
│ 176th St      │
│ 183rd St      │
│ 225th St      │
│ 231st St      │
│ 238th St      │
│ 242nd St      │
│    ·          │
│    ·          │
│    ·          │
│ 61st St       │
│ 80th St       │
│ 82nd St       │
│ 88th St       │
│ Beach 44th St │
│ Beach 90th St │
│ Beach 98th St │
│ Main St       │
│ St George     │
│ Stapleton     │
├───────────────┤
│   154 rows    │
│  (20 shown)   │
└───────────────┘

In [53]:
con.sql("""
SELECT * FROM cities WHERE country LIKE '_S_';
""").df()

,id,name,country,latitude,longitude,population,geom
0,7,Ramallah,PSE,31.90294,35.20621,24599,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,113,Palikir,FSM,6.91664,158.14997,4645,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,115,Agana,USA,13.47002,144.75002,122411,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,124,San Bernardino,USA,34.12038,-117.30003,1745000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,125,Bridgeport,USA,41.17998,-73.19996,1018000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
...,...,...,...,...,...,...,...
129,1187,Chicago,USA,41.82999,-87.75005,8990000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
130,1192,Madrid,ESP,40.40003,-3.68335,5567000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
131,1223,Los Angeles,USA,33.98998,-118.17998,12500000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
132,1224,Washington D.C.,USA,38.89955,-77.00942,4338000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


#### Filtering with Multiple Conditions

In [45]:
con1.sql("""
SELECT *
FROM nyc_subway_stations
WHERE BOROUGH = 'Brooklyn' AND NAME LIKE '%R%';
""").df()

,OBJECTID,ID,NAME,ALT_NAME,CROSS_ST,LONG_NAME,LABEL,BOROUGH,NGHBHD,ROUTES,TRANSFERS,COLOR,EXPRESS,CLOSED,geom
0,119.0,411.0,Bay Ridge Ave,None,4th Ave,Bay Ridge Ave (R) Brooklyn,Bay Ridge Ave (R),Brooklyn,None,R,R,YELLOW,None,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
1,124.0,388.0,Beverly Rd,None,E 16th St,Beverly Rd (Q) Brooklyn,Beverly Rd (Q),Brooklyn,None,Q,Q,YELLOW,None,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
2,125.0,96.0,Beverly Rd,None,Nostrand Ave,"Beverly Rd (2,5) Brooklyn","Beverly Rd (2,5)",Brooklyn,None,"2,5","2,5",RED-GREEN,None,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
3,145.0,389.0,Cortelyou Rd,None,E 16th St,Cortelyou Rd (Q) Brooklyn,Cortelyou Rd (Q),Brooklyn,None,Q,Q,YELLOW,None,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
4,201.0,396.0,Neck Rd,None,E 16th St,Neck Rd (Q) Brooklyn,Neck Rd (Q),Brooklyn,None,Q,Q,YELLOW,None,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
5,220.0,228.0,Ralph Ave,None,Fulton St,Ralph Ave (C) Brooklyn,Ralph Ave (C),Brooklyn,None,C,C,BLUE,None,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
6,221.0,229.0,Rockaway Ave,None,Fulton St,Rockaway Ave (C) Brooklyn,Rockaway Ave (C),Brooklyn,None,C,C,BLUE,None,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
7,222.0,87.0,Rockaway Ave,None,Livonia Ave,Rockaway Ave (3) Brooklyn,Rockaway Ave (3),Brooklyn,None,3,3,RED-GREEN,None,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
8,223.0,462.0,Rockaway Pky,Canarsie,Glenwood Rd,Rockaway Pky / Canarsie (L) Brooklyn,Rockaway Pky / Canarsie (L),Brooklyn,None,L,L,GREY,None,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."


The `HAVING` clause was added to SQL because the `WHERE` keyword could not be used with aggregate functions.


In [54]:
con.sql("""
SELECT COUNT(name), country
FROM cities
GROUP BY country
HAVING COUNT(name) > 40
ORDER BY COUNT(name) DESC;
""")

┌───────────────┬─────────┐
│ count("name") │ country │
│     int64     │ varchar │
├───────────────┼─────────┤
│           114 │ USA     │
│           100 │ CHN     │
│            81 │ RUS     │
│            69 │ IND     │
│            46 │ BRA     │
│            45 │ CAN     │
└───────────────┴─────────┘

#### Conditional statements

In [56]:
con.sql("""
SELECT name, population,
CASE
    WHEN population > 10000000 THEN 'Megacity'
    WHEN population > 1000000 THEN 'Large city'
    ELSE 'Small city'
END AS category
FROM cities;
""")

┌──────────────────┬────────────┬────────────┐
│       name       │ population │  category  │
│     varchar      │   int32    │  varchar   │
├──────────────────┼────────────┼────────────┤
│ Bombo            │      75000 │ Small city │
│ Fort Portal      │      42670 │ Small city │
│ Potenza          │      69060 │ Small city │
│ Campobasso       │      50762 │ Small city │
│ Aosta            │      34062 │ Small city │
│ Mariehamn        │      10682 │ Small city │
│ Ramallah         │      24599 │ Small city │
│ Vatican City     │        832 │ Small city │
│ Poitier          │      85960 │ Small city │
│ Clermont-Ferrand │     233050 │ Small city │
│   ·              │        ·   │    ·       │
│   ·              │        ·   │    ·       │
│   ·              │        ·   │    ·       │
│ Tokyo            │   35676000 │ Megacity   │
│ Mumbai           │   18978000 │ Megacity   │
│ Paris            │    9904000 │ Large city │
│ Santiago         │    5720000 │ Large city │
│ Kolkata    

#### Saving Results / Droping Tables

In [57]:
con.sql("CREATE TABLE cities2 AS SELECT name FROM cities;")

In [58]:
con.sql("FROM cities2;")

┌──────────────────┐
│       name       │
│     varchar      │
├──────────────────┤
│ Bombo            │
│ Fort Portal      │
│ Potenza          │
│ Campobasso       │
│ Aosta            │
│ Mariehamn        │
│ Ramallah         │
│ Vatican City     │
│ Poitier          │
│ Clermont-Ferrand │
│   ·              │
│   ·              │
│   ·              │
│ Tokyo            │
│ Mumbai           │
│ Paris            │
│ Santiago         │
│ Kolkata          │
│ Rio de Janeiro   │
│ Sao Paulo        │
│ Sydney           │
│ Singapore        │
│ Hong Kong        │
├──────────────────┤
│    1249 rows     │
│    (20 shown)    │
└──────────────────┘

In [59]:
con.sql("""
DROP TABLE IF EXISTS cities_usa;
CREATE TABLE cities_usa AS (SELECT * FROM cities WHERE country = 'USA');
""")

In [62]:
con.sql("FROM cities_usa;").df()

,id,name,country,latitude,longitude,population,geom
0,115,Agana,USA,13.47002,144.75002,122411,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,124,San Bernardino,USA,34.12038,-117.30003,1745000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,125,Bridgeport,USA,41.17998,-73.19996,1018000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,126,Rochester,USA,43.17043,-77.61995,755000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,160,International Falls,USA,48.60113,-93.41085,15240,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
...,...,...,...,...,...,...,...
109,1186,Atlanta,USA,33.83001,-84.39995,4506000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
110,1187,Chicago,USA,41.82999,-87.75005,8990000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
111,1223,Los Angeles,USA,33.98998,-118.17998,12500000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
112,1224,Washington D.C.,USA,38.89955,-77.00942,4338000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


Use the `INSERT INTO` statement to insert rows into a table.

In [63]:
con.sql("""
INSERT INTO cities_usa (SELECT * FROM cities WHERE country = 'CAN');
""")

In [64]:
con.sql("FROM cities_usa;").df()

,id,name,country,latitude,longitude,population,geom
0,115,Agana,USA,13.47002,144.75002,122411,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,124,San Bernardino,USA,34.12038,-117.30003,1745000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,125,Bridgeport,USA,41.17998,-73.19996,1018000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,126,Rochester,USA,43.17043,-77.61995,755000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,160,International Falls,USA,48.60113,-93.41085,15240,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
...,...,...,...,...,...,...,...
154,1002,Charlottetown,CAN,46.24928,-63.13133,42402,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
155,1171,Edmonton,CAN,53.55002,-113.49998,1058000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
156,1172,Montréal,CAN,45.50000,-73.58330,3678000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
157,1216,Vancouver,CAN,49.27342,-123.12164,2313328,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


#### Comments

In [66]:
con.sql("""
SELECT * FROM cities LIMIT 5 -- This is a comment;
""")

┌───────┬─────────────┬─────────┬──────────┬───────────┬────────────┬────────────────────────┐
│  id   │    name     │ country │ latitude │ longitude │ population │          geom          │
│ int32 │   varchar   │ varchar │  double  │  double   │   int32    │        geometry        │
├───────┼─────────────┼─────────┼──────────┼───────────┼────────────┼────────────────────────┤
│     1 │ Bombo       │ UGA     │   0.5833 │   32.5333 │      75000 │ POINT (32.5333 0.5833) │
│     2 │ Fort Portal │ UGA     │    0.671 │    30.275 │      42670 │ POINT (30.275 0.671)   │
│     3 │ Potenza     │ ITA     │   40.642 │    15.799 │      69060 │ POINT (15.799 40.642)  │
│     4 │ Campobasso  │ ITA     │   41.563 │    14.656 │      50762 │ POINT (14.656 41.563)  │
│     5 │ Aosta       │ ITA     │   45.737 │     7.315 │      34062 │ POINT (7.315 45.737)   │
└───────┴─────────────┴─────────┴──────────┴───────────┴────────────┴────────────────────────┘

In [67]:
con.sql("""
SELECT COUNT(name), country
FROM cities
/*
 * Adding Group by
 * Adding Order by
 */
GROUP BY country
ORDER BY COUNT(name) DESC
LIMIT 5;
""")

┌───────────────┬─────────┐
│ count("name") │ country │
│     int64     │ varchar │
├───────────────┼─────────┤
│           114 │ USA     │
│           100 │ CHN     │
│            81 │ RUS     │
│            69 │ IND     │
│            46 │ BRA     │
└───────────────┴─────────┘

### W3 Spatial SQL 1

#### Geometries

In [68]:
con.sql("""

CREATE or REPLACE TABLE samples (name VARCHAR, geom GEOMETRY);

/*
VARCHAR就是python里的str
*/

INSERT INTO samples VALUES
  ('Point', ST_GeomFromText('POINT(-100 40)')),
  ('Linestring', ST_GeomFromText('LINESTRING(0 0, 1 1, 2 1, 2 2)')),
  ('Polygon', ST_GeomFromText('POLYGON((0 0, 1 0, 1 1, 0 1, 0 0))')),
  ('PolygonWithHole', ST_GeomFromText('POLYGON((0 0, 10 0, 10 10, 0 10, 0 0),(1 1, 1 2, 2 2, 2 1, 1 1))')),
  ('Collection', ST_GeomFromText('GEOMETRYCOLLECTION(POINT(2 0),POLYGON((0 0, 1 0, 1 1, 0 1, 0 0)))'));

SELECT * FROM samples;

  """)

┌─────────────────┬───────────────────────────────────────────────────────────────────────┐
│      name       │                                 geom                                  │
│     varchar     │                               geometry                                │
├─────────────────┼───────────────────────────────────────────────────────────────────────┤
│ Point           │ POINT (-100 40)                                                       │
│ Linestring      │ LINESTRING (0 0, 1 1, 2 1, 2 2)                                       │
│ Polygon         │ POLYGON ((0 0, 1 0, 1 1, 0 1, 0 0))                                   │
│ PolygonWithHole │ POLYGON ((0 0, 10 0, 10 10, 0 10, 0 0), (1 1, 1 2, 2 2, 2 1, 1 1))    │
│ Collection      │ GEOMETRYCOLLECTION (POINT (2 0), POLYGON ((0 0, 1 0, 1 1, 0 1, 0 0))) │
└─────────────────┴───────────────────────────────────────────────────────────────────────┘

In [69]:
con.sql("SELECT name, ST_AsText(geom) AS geometry FROM samples;")

┌─────────────────┬───────────────────────────────────────────────────────────────────────┐
│      name       │                               geometry                                │
│     varchar     │                                varchar                                │
├─────────────────┼───────────────────────────────────────────────────────────────────────┤
│ Point           │ POINT (-100 40)                                                       │
│ Linestring      │ LINESTRING (0 0, 1 1, 2 1, 2 2)                                       │
│ Polygon         │ POLYGON ((0 0, 1 0, 1 1, 0 1, 0 0))                                   │
│ PolygonWithHole │ POLYGON ((0 0, 10 0, 10 10, 0 10, 0 0), (1 1, 1 2, 2 2, 2 1, 1 1))    │
│ Collection      │ GEOMETRYCOLLECTION (POINT (2 0), POLYGON ((0 0, 1 0, 1 1, 0 1, 0 0))) │
└─────────────────┴───────────────────────────────────────────────────────────────────────┘

In [70]:
con.sql("""

SELECT ST_X(geom), ST_Y(geom)
  FROM samples
  WHERE name = 'Point';

""")

┌────────────┬────────────┐
│ st_x(geom) │ st_y(geom) │
│   double   │   double   │
├────────────┼────────────┤
│     -100.0 │       40.0 │
└────────────┴────────────┘

In [71]:
con.sql("""

SELECT ST_Length(geom)
  FROM samples
  WHERE name = 'Linestring';

""")

┌───────────────────┐
│  st_length(geom)  │
│      double       │
├───────────────────┤
│ 3.414213562373095 │
└───────────────────┘

In [73]:
con.sql("""

SELECT name, ST_Area(geom)
  FROM samples
  WHERE name LIKE 'Polygon%';

""")

┌─────────────────┬───────────────┐
│      name       │ st_area(geom) │
│     varchar     │    double     │
├─────────────────┼───────────────┤
│ Polygon         │           1.0 │
│ PolygonWithHole │          99.0 │
└─────────────────┴───────────────┘

In [74]:
con1.sql("""
SELECT name, ST_GeometryType(geom), ST_Length(geom)
FROM nyc_streets
WHERE name = 'Pelham St';
""")

┌───────────┬───────────────────────┬───────────────────┐
│   NAME    │ st_geometrytype(geom) │  st_length(geom)  │
│  varchar  │     geometry_type     │      double       │
├───────────┼───────────────────────┼───────────────────┤
│ Pelham St │ MULTILINESTRING       │ 50.32314951660229 │
└───────────┴───────────────────────┴───────────────────┘

In [75]:
con1.sql("""
SELECT name, ST_AsGeoJSON(geom), ST_Length(geom)
FROM nyc_subway_stations
WHERE name = 'Broad St';
""")

┌──────────┬──────────────────────────────────────────────────────────────────────┬─────────────────┐
│   NAME   │                          st_asgeojson(geom)                          │ st_length(geom) │
│ varchar  │                                 json                                 │     double      │
├──────────┼──────────────────────────────────────────────────────────────────────┼─────────────────┤
│ Broad St │ {"type":"Point","coordinates":[583571.9059213118,4506714.341192182]} │             0.0 │
└──────────┴──────────────────────────────────────────────────────────────────────┴─────────────────┘

What is the most westerly subway station?

In [76]:
con1.sql("""
SELECT name, ST_X(geom)
FROM nyc_subway_stations
ORDER BY ST_X(geom) ASC
LIMIT 1;
""")

┌─────────────┬───────────────────┐
│    NAME     │    st_x(geom)     │
│   varchar   │      double       │
├─────────────┼───────────────────┤
│ Tottenville │ 563292.1172580556 │
└─────────────┴───────────────────┘

#### Spatial Relationships

In [77]:
con1.sql("""
SELECT ST_Distance(
  ST_GeomFromText('POINT(0 5)'),
  ST_GeomFromText('LINESTRING(-2 2, 2 2)')) as dist;
""")

┌────────┐
│  dist  │
│ double │
├────────┤
│    3.0 │
└────────┘

In [78]:
con1.sql("""
SELECT name
FROM nyc_streets
WHERE ST_DWithin(
        geom,
        ST_GeomFromText('POINT(583571 4506714)'),
        10
      );
""")

┌───────────┐
│   NAME    │
│  varchar  │
├───────────┤
│ Wall St   │
│ Broad St  │
│ Nassau St │
└───────────┘

In [80]:
con1.sql("""
SELECT Sum(popn_total)
  FROM nyc_census_blocks
  WHERE ST_DWithin(
   geom,
   ST_GeomFromText('LINESTRING(586782 4504202,586864 4504216)'),
   50
  );
  """)

┌─────────────────┐
│ sum(popn_total) │
│     int128      │
├─────────────────┤
│            1438 │
└─────────────────┘

## W4 Spatial SQL 2

What subway station is in \'Little Italy\'? What subway route is it on?

In [81]:
con1.sql("""
SELECT s.name, s.routes
FROM nyc_neighborhoods n
JOIN nyc_subway_stations s
ON ST_Contains(n.geom, s.geom)
WHERE n.name = 'Little Italy'
""")

┌───────────┬─────────┐
│   NAME    │ ROUTES  │
│  varchar  │ varchar │
├───────────┼─────────┤
│ Spring St │ 6       │
└───────────┴─────────┘

What are all the neighborhoods served by the 6-train?

In [82]:
con1.sql("""
SELECT DISTINCT n.name, n.boroname
FROM nyc_neighborhoods n
JOIN nyc_subway_stations s
ON ST_Contains(n.geom, s.geom)
WHERE strpos(s.routes,'6')>0;
""")

┌────────────────────┬───────────┐
│        NAME        │ BORONAME  │
│      varchar       │  varchar  │
├────────────────────┼───────────┤
│ Gramercy           │ Manhattan │
│ Mott Haven         │ The Bronx │
│ Murray Hill        │ Manhattan │
│ Soundview          │ The Bronx │
│ Greenwich Village  │ Manhattan │
│ Financial District │ Manhattan │
│ Parkchester        │ The Bronx │
│ Little Italy       │ Manhattan │
│ Chinatown          │ Manhattan │
│ Midtown            │ Manhattan │
│ Upper East Side    │ Manhattan │
│ Hunts Point        │ The Bronx │
│ East Harlem        │ Manhattan │
│ South Bronx        │ The Bronx │
│ Yorkville          │ Manhattan │
├────────────────────┴───────────┤
│ 15 rows              2 columns │
└────────────────────────────────┘

After 9/11, the \'Battery Park\' neighborhood was off limits for several days. How many people had to be evacuated?

In [83]:
con1.sql("""
SELECT SUM(c.POPN_TOTAL)
FROM nyc_census_blocks c
JOIN nyc_neighborhoods n
ON ST_Intersects(n.geom, c.geom)
WHERE n.name = 'Battery Park'
""")
#只要 SELECT 里有 SUM / COUNT / AVG / MAX / MIN：
#其他被 SELECT 出来的字段
#必须 出现在 GROUP BY 里
#或者被包在聚合函数里

┌───────────────────┐
│ sum(c.POPN_TOTAL) │
│      int128       │
├───────────────────┤
│             17153 │
└───────────────────┘

What neighborhood has the highest population density (persons/km2)?

In [84]:
con1.sql("""
SELECT n.name, SUM(c.POPN_TOTAL)/ST_Area(n.geom)*1000000 AS density
FROM nyc_census_blocks c
JOIN nyc_neighborhoods n
ON ST_Intersects(n.geom, c.geom)
GROUP BY n.name, n.geom
ORDER BY density DESC
LIMIT 1
""")

┌───────────────────┬──────────────────┐
│       NAME        │     density      │
│      varchar      │      double      │
├───────────────────┼──────────────────┤
│ North Sutton Area │ 68435.1328377268 │
└───────────────────┴──────────────────┘